In [ ]:
pip install --upgrade "ibm-watson>=6.1.0"

In [1]:
# Dados da api
nlu_apikey = "Sua apiKey aqui"
nlu_url = "Sua url aqui

In [2]:
# Criando os comentários para ser usado na Api
dados = {'comentarios':['Péssimo app não atende o esperado', 'Não gostei horrível', 'Adorei atende minhas necessidades', 'Lixo trava toda hora', 'Interessante tem grande potencial', 'Confesso que não uso muito', 'O app tem muita instabilidade', 'Estou tendo problemas com o uso do app', 'Muito bom excelente', 'Gostei muito achei perfeito','Lamentável app não atende o esperado', 'Gostei supera os demais apps', 'Adorei atende minhas necessidades', 'Que coisa ruim trava toda hora', 'Tem grande potencial', 'Não uso muito', 'O app tem muita falha', 'Não tenho problemas com o uso do app', 'Muito bom excelente', 'Odiei nem abre']}

In [18]:
# Importando as bibliotecas necessárias
import json
from flask import Flask, jsonify, request
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, ConceptsOptions, RelationsOptions, EmotionOptions, EntitiesOptions, SemanticRolesOptions, SentimentOptions, CategoriesOptions, SyntaxOptions, SyntaxOptionsTokens

In [19]:
# Autenticando a chave
authenticator = IAMAuthenticator(nlu_apikey)
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2023-06-23',
    authenticator=authenticator
)
natural_language_understanding.set_service_url(nlu_url)

In [20]:
# Analisando os positivos e negativos e adcionando nas respectivas listas
positivos = list()
negativos = list()

for i in range(len(dados['comentarios'])):
    response = natural_language_understanding.analyze(text=dados['comentarios'][i], language='pt', features=Features(sentiment=SemanticRolesOptions())).get_result(),
    
    for sent in response:
        if (sent['sentiment']['document']['label']) == 'positive':
            positivos.append(dados['comentarios'][i])
        else:
            negativos.append(dados['comentarios'][i])


In [21]:
# Comentarios com substantitvos positivos
subs_positivos = list()

for subP in range(len(positivos)):
    response_subP = natural_language_understanding.analyze(text=positivos[subP], 
                                                          features=Features(syntax=SyntaxOptions(sentences=True, 
                                                          tokens=SyntaxOptionsTokens(lemma=True, part_of_speech=True)))).get_result(),
    for subsP in response_subP:
        if (subsP['syntax']['tokens'][2]['part_of_speech']) == 'NOUN':
            subs_positivos.append(subsP['syntax']['tokens'][2]['text'])
result_subP = (f'Total(is): {len(subs_positivos)}, Comentário(s): {sorted(subs_positivos)}')

In [22]:
# Comentarios com substantitvos negativos
subs_negativos = list()

for subN in range(len(negativos)):
    response_subN = natural_language_understanding.analyze(text=negativos[subN], language='pt', 
                                                          features=Features(syntax=SyntaxOptions(sentences=True, 
                                                          tokens=SyntaxOptionsTokens(lemma=True, part_of_speech=True)))).get_result(),
    for subsN in response_subN:
        if (subsN['syntax']['tokens'][2]['part_of_speech']) == 'NOUN':
            subs_negativos.append(subsN['syntax']['tokens'][2]['text'])
result_subN= (f'Total(is): {len(subs_negativos)}, Comentário(s): {sorted(subs_negativos)}')    

In [33]:
# Exibindo os resultados
print(f'A lista dos comentários POSITIVOS: {positivos}\n')
print(f'A Lista dos comentários NEGATIVOS: {negativos}\n')
print(f'Total de substantivos POSITIVOS identificados: {result_subP}\n')
print(f'Total de substantivos NEGATIVOS identificados: {result_subN}')

A lista dos comentários POSITIVOS: ['Adorei atende minhas necessidades', 'Interessante tem grande potencial', 'Muito bom excelente', 'Gostei muito achei perfeito', 'Gostei supera os demais apps', 'Adorei atende minhas necessidades', 'Tem grande potencial', 'Muito bom excelente']

A Lista dos comentários NEGATIVOS: ['Péssimo app não atende o esperado', 'Não gostei horrível', 'Lixo trava toda hora', 'Confesso que não uso muito', 'O app tem muita instabilidade', 'Estou tendo problemas com o uso do app', 'Lamentável app não atende o esperado', 'Que coisa ruim trava toda hora', 'Não uso muito', 'O app tem muita falha', 'Não tenho problemas com o uso do app', 'Odiei nem abre']

Total de substantivos POSITIVOS identificados: Total(is): 1, Comentário(s): ['potencial']

Total de substantivos NEGATIVOS identificados: Total(is): 2, Comentário(s): ['problemas', 'problemas']
